### Create a Voila app for monitoring the market with customizable parameters for the underlying asset, start date, and end date

Retrieve Market Data Function: Create a Python function to fetch data from Yahoo Finance. This function should take the underlying symbol, start date, and end date as input parameters.

Voila App Structure: Set up a Voila app with an interactive interface. The first line of the app will contain input widgets for the user to customize the underlying symbol, start date, and end date.

Tabs for Returns: Implement tabs in the app to display daily, weekly, monthly, and year-to-date (YTD) returns. Each tab will show a chart or a table representing the respective returns.

Automatic Refresh: Ensure that the tabs and charts are automatically refreshed when any of the input parameters (underlying, start date, end date) are changed.

Deployment: Deploy the app using Voila.

Let's start by creating the Python function to fetch market data. We will use yfinance, a popular library for fetching historical market data from Yahoo Finance.

In [10]:
import yfinance as yf
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import datetime
import qgrid
import pandas as pd

In [11]:
def calculate_daily_returns(data):
    return data['Close'].pct_change()

def calculate_weekly_returns(data):
    return data['Close'].resample('W').ffill().pct_change()

def calculate_monthly_returns(data):
    return data['Close'].resample('M').ffill().pct_change()

def calculate_ytd_returns(data):
    year_start = data.index[0].year
    # Get data starting from the beginning of the year
    start_of_year_data = data[f'{year_start}-01-01':]

    # Check if the data is empty
    if not start_of_year_data.empty:
        start_price = start_of_year_data['Close'].iloc[0]
        return (data['Close'] / start_price) - 1
    else:
        # Return an empty Series or handle as needed
        return pd.Series([])

In [12]:
# Define Function to Fetch Data with Date Range
def fetch_stock_data(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)
    return stock.history(start=start_date, end=end_date)

def plot_stock_data(data):
    plt.figure(figsize=(10, 5))
    plt.plot(data['Close'], label='Close Price')
    plt.title('Stock Price Over Time')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.show()

In [13]:
def style_dataframe(df):
    # Convert Series to DataFrame if necessary
    if isinstance(df, pd.Series):
        df = df.to_frame()

    # Apply styling
    return df.style.set_table_styles(
        [{
            'selector': 'thead',
            'props': [
                ('background-color', '#606060'),
                ('color', 'white'),
                ('font-weight', 'bold')
            ]
        }]
    ).set_properties(**{
        'background-color': '#f4f4f4',
        'color': 'black',
        'border-color': 'black',
        'border-width': '1px',
        'border-style': 'solid'
    }).format("{:.2%}")


In [14]:
# Other imports and function definitions remain the same
import ipywidgets as widgets

# Example filter widgets
filter_dropdown = widgets.Dropdown(
    options=['All', 'Positive', 'Negative'],
    value='All',
    description='Filter:',
)

def filter_data(data, filter_value):
    if filter_value == 'Positive':
        return data[data > 0]
    elif filter_value == 'Negative':
        return data[data < 0]
    return data

def update_filtered_data(b):
    selected_filter = filter_dropdown.value
    # Assuming 'original_data' is your unfiltered DataFrame
    filtered_data = filter_data(original_data, selected_filter)
    # Now update the display with 'filtered_data'


# Create Interactive Widgets
ticker_input = widgets.Text(value='AAPL', description='Ticker:', disabled=False)

start_date_picker = widgets.DatePicker(description='Start Date', value=datetime.date.today() - datetime.timedelta(days=365))
end_date_picker = widgets.DatePicker(description='End Date', value=datetime.date.today())

button = widgets.Button(description="Plot Data")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        ticker = ticker_input.value
        start_date = start_date_picker.value
        end_date = end_date_picker.value
        data = fetch_stock_data(ticker, start_date, end_date)
        plot_stock_data(data)
        
        # Create tabs for different performances
        tab = widgets.Tab()
        tab_titles = ['Daily', 'Weekly', 'Monthly', 'YTD']
        returns_functions = [calculate_daily_returns, calculate_weekly_returns, calculate_monthly_returns, calculate_ytd_returns]
        tab.children = [widgets.Output() for _ in tab_titles]

        for i, title in enumerate(tab_titles):
            tab.set_title(i, title)
            returns = returns_functions[i](data).dropna()
            with tab.children[i]:
                clear_output(wait=True)
                # Display styled DataFrame
                styled_df = style_dataframe(returns)
                display(styled_df)

        display(tab)


button.on_click(on_button_clicked)

# Display everything
display(ticker_input, start_date_picker, end_date_picker, button, output)


Text(value='AAPL', description='Ticker:')

DatePicker(value=datetime.date(2023, 1, 11), description='Start Date')

DatePicker(value=datetime.date(2024, 1, 11), description='End Date')

Button(description='Plot Data', style=ButtonStyle())

Output()

ValueError: year 20222 is out of range